In [1]:
import tensorflow as tf
import numpy as np
import utils

from collections import Counter


__author__ = "Olivares Castillo José Luis"

#tf.enable_eager_execution()
tf.logging.set_verbosity(tf.logging.INFO)

print("TensorFlow version: {}".format(tf.VERSION))
#print("Eager execution: {}".format(tf.executing_eagerly()))

if tf.test.gpu_device_name():
    print("GPU disponible")

/home/olivares/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow version: 1.7.0


In [2]:
words_scr_lexicon, words_trg_lexicon = utils.get_lexicon("en-it.test")
print(len(words_scr_lexicon), len(words_trg_lexicon))

1869 1869


In [3]:
source_vec = utils.open_file('en.fst')
words_src, source_vec = utils.read(source_vec,is_zipped=False)
eval_src = list(set(words_scr_lexicon))
src_vec = utils.get_vectors(eval_src, words_src, source_vec)
print(src_vec.shape)

(1500, 300)


In [4]:
target_vec = utils.open_file("it.fst")
words_trg, target_vec = utils.read(target_vec,is_zipped=False)
#eval_it = list(set(it))
#trg_vec = get_vectors(eval_it, words_it, it_vec)
print(target_vec.shape)

(199996, 300)


In [5]:
test_vectors = src_vec

In [6]:
tf.reset_default_graph()
sess = tf.Session()
path="models/en-it/1/"
saver = tf.train.import_meta_graph(path+"model2250.ckpt.meta")
saver.restore(sess,tf.train.latest_checkpoint(path))

INFO:tensorflow:Restoring parameters from models/en-it/1/model2250.ckpt


In [7]:
graph = tf.get_default_graph()
X = graph.get_tensor_by_name("input/input_es:0")
kprob = graph.get_tensor_by_name("dropout_prob:0")

In [8]:
([n.name for n in graph.as_graph_def().node])

['input/input_es',
 'input/target_na',
 'dropout_prob',
 'h1/kernel/Initializer/truncated_normal/shape',
 'h1/kernel/Initializer/truncated_normal/mean',
 'h1/kernel/Initializer/truncated_normal/stddev',
 'h1/kernel/Initializer/truncated_normal/TruncatedNormal',
 'h1/kernel/Initializer/truncated_normal/mul',
 'h1/kernel/Initializer/truncated_normal',
 'h1/kernel',
 'h1/kernel/Assign',
 'h1/kernel/read',
 'h1/bias/Initializer/zeros/shape_as_tensor',
 'h1/bias/Initializer/zeros/Const',
 'h1/bias/Initializer/zeros',
 'h1/bias',
 'h1/bias/Assign',
 'h1/bias/read',
 'h1/MatMul',
 'h1/BiasAdd',
 'h1/Elu',
 'dropout/Identity',
 'nah_predicted/kernel/Initializer/truncated_normal/shape',
 'nah_predicted/kernel/Initializer/truncated_normal/mean',
 'nah_predicted/kernel/Initializer/truncated_normal/stddev',
 'nah_predicted/kernel/Initializer/truncated_normal/TruncatedNormal',
 'nah_predicted/kernel/Initializer/truncated_normal/mul',
 'nah_predicted/kernel/Initializer/truncated_normal',
 'nah_predi

In [16]:
output_NN = graph.get_tensor_by_name("nah_predicted/BiasAdd:0")
#output_NN = graph.get_tensor_by_name("nah_predicted:0")
#code = graph.get_tensor_by_name("xw_plus_b_2:0")
print(output_NN)

feed_dict = {X: test_vectors, kprob: 1}
pred = sess.run(output_NN, feed_dict)
print(pred.shape)

Tensor("nah_predicted/BiasAdd:0", shape=(?, 300), dtype=float64)
(1500, 300)


In [17]:
%%time
top_10 = [utils.get_top10_vectors(pred[_], target_vec) for _ in range(pred.shape[0])]

CPU times: user 5min 28s, sys: 4min 15s, total: 9min 43s
Wall time: 3min 39s


In [18]:
%%time
closest = [utils.closest_word_to(top_10[_], words_trg) for _ in range(pred.shape[0])]

CPU times: user 14 ms, sys: 1.42 ms, total: 15.4 ms
Wall time: 168 ms


In [19]:
resultados = {palabra_en: top_10_it for (palabra_en, top_10_it) in zip(eval_src, closest)}

In [20]:
gold = utils.gold_dict(words_scr_lexicon, words_trg_lexicon)

In [21]:
%%time
p1, p5, p10 = 0, 0, 0
list_en_eval = list(resultados.keys())
hits, not_found = [], []

for palabra_gold in list_en_eval:
    for i in gold[palabra_gold]:
        if i in resultados[palabra_gold]:
            hits.append(resultados[palabra_gold].index(i))
    if hits.__len__() > 0:
        if min(hits) == 0:
            p1 += 1
            p5 += 1
            p10 += 1
        if min(hits) >= 1 and min(hits) <= 5:
            p5 += 1
            p10 += 1
        if min(hits) > 5 and min(hits) < 10:
            p10 += 1
    else:
        not_found.append(palabra_gold)
    hits.clear()

length = list_en_eval.__len__()
print("not found:", not_found.__len__(), "-", not_found.__len__() / length, "%")
print("P@1:", p1, "\tP@5:", p5, "\tP@10:", p10)
print("P@1:", p1 / length, "\tP@5:", p5 /length, "\tP@10:", p10 / length)

not found: 671 - 0.44733333333333336 %
P@1: 505 	P@5: 769 	P@10: 829
P@1: 0.33666666666666667 	P@5: 0.5126666666666667 	P@10: 0.5526666666666666
CPU times: user 4.05 ms, sys: 28 µs, total: 4.07 ms
Wall time: 3.96 ms
